In [37]:
begin
	using Flux
	using Flux: onehot
	using Flux: gradient
	using Flux.Optimise: update!
	using Flux: onecold
	using CUDA
	using Transformers
	using Transformers.Basic 
    using Transformers.Datasets: batched
    enable_gpu(false) 
end
	

false

In [38]:
#Labeling: vielleicht ändern

begin
	labels = map(string, 1:10)
	startsym = "11"
	endsym = "12"
	unksym = "0"
	labels = [unksym, startsym, endsym, labels...]
	vocab = Vocabulary(labels, unksym)
end

Vocabulary{String}(13, unk=0)

In [39]:
#function for generate training datas 

#nichts ändern
sample_data() = (d = map(string, rand(1:10, 10)); (d,d))

sample_data (generic function with 1 method)

In [40]:
#nichts ändern

sample_data()

(["10", "1", "6", "10", "1", "3", "4", "1", "3", "5"], ["10", "1", "6", "10", "1", "3", "4", "1", "3", "5"])

In [41]:
#function for adding start & end symbol

#nichts ändern
preprocess(x) = [startsym, x..., endsym]

preprocess (generic function with 1 method)

In [42]:
#nichts ändern

begin
    @show sample_ex = preprocess.(sample_data())
    @show encoded_sample_ex = vocab(sample_ex[1]) #use Vocabulary to encode the training data
    end

sample_ex = preprocess.(sample_data()) = (["11", "8", "6", "8", "3", "5", "4", "5", "7", "7", "6", "12"], ["11", "8", "6", "8", "3", "5", "4", "5", "7", "7", "6", "12"])
encoded_sample_ex = vocab(sample_ex[1]) = [2, 11, 9, 11, 6, 8, 7, 8, 10, 10, 9, 3]


12-element Vector{Int64}:
  2
 11
  9
 11
  6
  8
  7
  8
 10
 10
  9
  3

In [43]:
#nichts ändern
sample = preprocess.(sample_data())

(["11", "10", "4", "5", "3", "1", "1", "8", "4", "10", "5", "12"], ["11", "10", "4", "5", "3", "1", "1", "8", "4", "10", "5", "12"])

In [44]:
#nichts ändern
encoded_sample = vocab(sample[1])

12-element Vector{Int64}:
  2
 13
  7
  8
  6
  4
  4
 11
  7
 13
  8
  3

In [45]:
#define a Word embedding layer which turn word index to word vector

#embeding vielleicht ändern
embed = Embed(512, length(vocab)) |> gpu

Embed(512)

In [46]:
#define a position embedding layer metioned above

#embeding vielleicht ändern
pe = PositionEmbedding(512) |> gpu

PositionEmbedding(512)

In [47]:
#wrapper for get embedding

#embeding vielleicht ändern
function embedding(x)
    we = embed(x, inv(sqrt(512)))
    e = we .+ pe(we)
    return e
  end

embedding (generic function with 1 method)

In [48]:
#define 2 layer of transformer

#layer auf jeden Fall ändern
encode_t1 = Transformer(512, 8, 64, 2048) |> gpu

Transformer(head=8, head_size=64, pwffn_size=2048, size=512)

In [49]:
encode_t2 = Transformer(512, 8, 64, 2048) |> gpu

Transformer(head=8, head_size=64, pwffn_size=2048, size=512)

In [50]:
encode_t3 = Transformer(512, 8, 64, 2048) |> gpu

Transformer(head=8, head_size=64, pwffn_size=2048, size=512)

In [51]:
encode_t4 = Transformer(512, 8, 64, 2048) |> gpu

Transformer(head=8, head_size=64, pwffn_size=2048, size=512)

In [52]:
#define 2 layer of transformer decoder

#layer auf jeden Fall ändern
decode_t1 = TransformerDecoder(512, 8, 64, 2048) |> gpu

TransformerDecoder(head=8, head_size=64, pwffn_size=2048, size=512)

In [53]:
decode_t2 = TransformerDecoder(512, 8, 64, 2048) |> gpu

TransformerDecoder(head=8, head_size=64, pwffn_size=2048, size=512)

In [54]:
decode_t3 = TransformerDecoder(512, 8, 64, 2048) |> gpu

TransformerDecoder(head=8, head_size=64, pwffn_size=2048, size=512)

In [55]:
decode_t4 = TransformerDecoder(512, 8, 64, 2048) |> gpu

TransformerDecoder(head=8, head_size=64, pwffn_size=2048, size=512)

In [56]:
#define the layer to get the final output probabilities

#ÄNDERN
linear = Positionwise(Dense(512, length(vocab)), logsoftmax) |> gpu

Positionwise(Dense(512, 13), logsoftmax)

In [57]:
#neue Layer einfügen
function encoder_forward(x)
    e = embedding(x)
    t1 = encode_t1(e)
    t2 = encode_t2(t1)
    t3 = encode_t2(t2)
    t4 = encode_t2(t3)
    return t2
  end

encoder_forward (generic function with 1 method)

In [58]:
#neue Layer einfügen
function decoder_forward(x, m)
    e = embedding(x)
    t1 = decode_t1(e, m)
    t2 = decode_t2(t1, m)
    t3 = decode_t3(t2, m)
    t4 = decode_t4(t3, m)
    p = linear(t4)
    return p
  end

decoder_forward (generic function with 1 method)

In [59]:
#bleibt so
enc = encoder_forward(encoded_sample)

512×12 Matrix{Float32}:
 -0.783976   -1.44352    -1.62487   …  -1.28395   -0.714117   -0.155425
 -0.146617   -0.010778   -0.516757     -1.06788   -1.39345    -1.26417
 -1.2381     -1.95535    -2.3793       -2.29065   -1.87061    -1.46151
 -1.53198    -1.34922    -1.72902      -2.11023   -2.55284    -2.53758
 -2.21752    -2.72517    -3.14104      -3.4639    -3.23391    -2.70198
  0.762421    0.861737    0.652185  …   0.408097  -0.225156   -0.490134
  0.758689    0.183248   -0.424401     -0.486083  -0.604298   -0.292827
 -0.463856   -0.339417   -0.691884     -0.385156  -0.945159   -1.60126
  0.604982    0.176191   -0.381225     -0.226197  -0.481422   -0.209374
  1.5348      1.58561     1.22508       2.06731    1.51448     0.97457
  ⋮                                 ⋱              ⋮          
 -1.67825    -1.65809    -1.61248      -1.62557   -1.77096    -1.88959
 -2.58246    -2.65191    -2.70075      -2.49049   -2.41351    -2.49079
  1.21626     1.28922     1.14408   …   1.3474     1.0527

In [60]:
#bleibt so
probs = decoder_forward(encoded_sample, enc)

13×12 Matrix{Float32}:
 -3.19656   -3.22723   -3.23104   …  -3.2662    -3.2207    -3.1706
 -2.88185   -2.8212    -2.8489       -2.78626   -2.79146   -2.80963
 -2.00394   -2.06255   -2.06924      -2.10291   -2.02452   -2.00704
 -3.392     -3.36367   -3.32679      -3.43687   -3.42461   -3.39496
 -0.966129  -0.950526  -0.944796     -0.961614  -0.975533  -0.989869
 -2.96368   -3.05272   -3.15992   …  -3.08168   -3.08212   -3.03138
 -3.52724   -3.48012   -3.49771      -3.66726   -3.70329   -3.68949
 -3.52085   -3.4346    -3.42959      -3.6459    -3.69418   -3.69212
 -4.46938   -4.54143   -4.59207      -4.64139   -4.59683   -4.52108
 -5.1196    -5.10336   -5.16981      -5.26149   -5.28434   -5.2968
 -2.43192   -2.42796   -2.40442   …  -2.29      -2.28854   -2.2703
 -2.60136   -2.59534   -2.56446      -2.62448   -2.64536   -2.65356
 -2.74877   -2.76971   -2.75775      -2.55677   -2.60085   -2.65182

In [61]:
#hier vielleicht etwas ändern --> herausfinden, was diese funktion macht
function smooth(et)
    sm = fill!(similar(et, Float32), 1e-6/size(embed, 2))
    p = sm .* (1 .+ -et)
    label = p .+ et .* (1 - convert(Float32, 1e-6))
    label
end

smooth (generic function with 1 method)

In [62]:
Flux.@nograd smooth

In [63]:
#define loss function

#ÄNDERN
function loss(x, y)
    label = onehot(vocab, y) #turn the index to one-hot encoding
    label = smooth(label) #perform label smoothing
    enc = encoder_forward(x)
    probs = decoder_forward(y, enc)
    l = logkldivergence(label[:, 2:end, :], probs[:, 1:end-1, :])  #erstmal so lassen
    return l
  end

loss (generic function with 1 method)

In [64]:
#collect all the parameters

#Layer neu einfügen
ps = params(embed, pe, encode_t1, encode_t2, encode_t3, encode_t4, decode_t1, decode_t2, decode_t3, decode_t4, linear)

Params([Float32[-0.72727716 -0.5593858 … -0.9596893 0.44172972; -0.97853994 0.36645007 … -0.25209206 -0.44719723; … ; 0.5352608 0.38274774 … -0.48243105 -0.30066285; 0.3348228 1.0622799 … -0.9543995 -2.6325977], Float32[0.5403023 -0.41614684 … 0.4000682 0.9873536; 0.8218562 0.9364147 … 0.37902638 0.97646356; … ; 1.0 1.0 … 0.9943822 0.99437124; 0.0001 0.0002 … 0.10212166 0.10222114], Float32[0.041380215 0.017902676 … -0.06163475 0.043483708; 0.038517594 0.03275342 … 0.02001686 0.056871176; … ; 0.06690922 -0.015104914 … -0.00750591 -0.05997853; -0.06495662 -0.028793307 … -0.06466323 0.037790034], Float32[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], Float32[0.02827442 -0.046335597 … -0.028715653 -0.05698261; 0.040791266 -0.018646423 … 0.052706953 -0.05531201; … ; 0.048989788 0.010275421 … -0.060929928 -0.01857074; 0.023648873 0.07268102 … 0.033567578 -0.06601943], Float32[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0

In [65]:
opt = ADAM(1e-4) #bleibt

ADAM(0.0001, (0.9, 0.999), IdDict{Any, Any}())

In [66]:
#define training loop

#ÄNDERN siehe Paper Kapitel 4.1
function train!()
    @info "start training"
    for i = 1:1000
      data = batched([sample_data() for i = 1:32]) #create 32 random sample and batched
      x, y = preprocess.(data[1]), preprocess.(data[2])
      x, y = vocab(x), vocab(y) #encode the data
      x, y = todevice(x, y) #move to gpu
      grad = gradient(()->loss(x, y), ps)
      if i % 8 == 0
          l = loss(x, y)
          println("loss = $l")
      end
      update!(opt, ps, grad)
    end
  end

train! (generic function with 1 method)

In [67]:
train!()

┌ Info: start training
└ @ Main /Users/johannes/Documents/GitHub/DM2022-LinearTransformers/Templates/Exercise11.ipynb:3


loss = 85.28133


loss = 77.54999


loss = 77.819855


loss = 77.73625


...

loss = 0.002265683


loss = 0.0039245673


loss = 0.0032930141


loss = 0.0503941


loss = 0.049990732


In [68]:
#erst einmal so lassen, vielleicht einzelne Änderungen
function translate(x)
    ix = todevice(vocab(preprocess(x)))
    seq = [startsym]

    enc = encoder_forward(ix)

    len = length(ix)
    for i = 1:2len
        trg = todevice(vocab(seq))
        dec = decoder_forward(trg, enc)
        #move back to gpu due to argmax wrong result on CuArrays
        ntok = onecold(collect(dec), labels)
        push!(seq, ntok[end])
        ntok[end] == endsym && break
    end
  seq[2:end-1]
end

translate (generic function with 1 method)

In [69]:
translate(map(string, [5,5,6,6,1,12,3,4,6]))

10-element Vector{String}:
 "5"
 "5"
 "6"
 "6"
 "1"
 "6"
 "3"
 "4"
 "6"
 "6"